In [1]:
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import re

from pyarrow.dataset import HivePartitioning

from model.InpatientsModel import InpatientsModel

Select a particular dataset and model run, then create an instance of the Inpatients Model class and load the datset.

In [2]:
dataset = "data/synthetic"
model_run = f"test/20220110_104353"
m = InpatientsModel(f"{dataset}/results/{model_run}")
ip = pq.read_pandas(f"{dataset}/ip.parquet").to_pandas()
ip.drop(["hsagrp"], axis = "columns", inplace = True)

We can now perform a model run. The `0` is the model run number. This alters the random seed used for the PRNG, meaning each time we run the model with this value we will get the same results.

In [3]:
selected_variant, results = m.run(0)
results

,rn,speldur,classpat,admission_avoidance_strategy,los_reduction_strategy
0,69912,0.0,2,NULL,NULL
1,69912,0.0,2,NULL,NULL
2,22261,0.0,2,NULL,improved_discharge_planning_elective
3,121349,1.0,1,NULL,improved_discharge_planning_non-elective
4,121349,1.0,1,NULL,improved_discharge_planning_non-elective
...,...,...,...,...,...
147702,121847,0.0,1,NULL,improved_discharge_planning_non-elective
147703,47388,0.0,2,NULL,improved_discharge_planning_elective
147704,47388,0.0,2,NULL,improved_discharge_planning_elective
147705,78737,1.0,1,NULL,NULL


We can join the results back to the full dataset to get all columns.

In [4]:
merged_results = ip.drop(["classpat", "speldur"], axis = "columns").merge(results, on = "rn")
merged_results.shape

(147707, 15)

We can now compare the base data to the modelled results.

In [5]:
# mean length of stay in the baseline vs results
(np.mean(ip.speldur), np.mean(merged_results.speldur))

(2.1196528833328165, 1.7333707948844672)

In [6]:
# number of rows in the baseline vs results
(len(ip.index), len(merged_results.index))

(128948, 147707)

In [7]:
# count by patient classification.
# Note: -1 indicates that a row has moved from the inpatients dataset to outpatients
pd.merge(
  ip.value_counts("classpat").to_frame("baseline"),
  merged_results.value_counts("classpat").to_frame("results"),
  left_index = True,
  right_index = True,
  how = "outer"
).fillna(0)

,baseline,results
classpat,,
-1,0.0,261
1,69560.0,76510
2,47864.0,57186
3,10476.0,12659
4,9.0,8
5,1039.0,1083


We need to filter out the `classpat == -1` cases, these will be merged into Outpatients.

In [8]:
outpatients_rows = merged_results.classpat == "-1"

outpatients_results = merged_results[outpatients_rows] \
  [["rn", "age", "sex", "ethnos", "imd04_decile", "tretspef", "admidate"]] \
  .rename(columns = { "admidate": "apptdate" })

merged_results = merged_results[~outpatients_rows]
merged_results.shape

(147446, 15)

Now we need to do a bit of data cleansing, the modelling process updates the `speldur` (length of stay), but doesn't change the `disdate`.

In [9]:
merged_results.disdate = pd.to_datetime(merged_results.admidate) + pd.to_timedelta(merged_results.speldur, 'd')

## Multiple Model Runs

We can run multiple iterations of the model using the `multi_model_runs()` function. This function run's slightly differently from `run()` in two specific ways:

1. It tries to run in parallel across multiple CPU cores, though this tend's to work much better on Linux than Windows (dependent on how this is run in Windows it may be stuck using a single CPU no matter how many you ask for)
2. Instead of returning the results, it immediately saves them to disk in the folder `results/[model_name]/[model_run_date]/results`

In [10]:
cpu_count = os.cpu_count()
m.multi_model_runs(0, cpu_count * 3, cpu_count)

36 ip model runs on 12 cpus


100%|██████████| 36/36 [00:11<00:00,  3.07it/s]


Timings:
* model runs:    mean: 1.273, range: [1.015, 1.585]
* save results:  mean: 0.165, range: [0.094, 0.261]
* total elapsed: mean: 1.438, range: [1.109, 1.759]


In [11]:
partitioning = HivePartitioning(pa.schema([("model_run", pa.int32())]))

path = f"{dataset}/results/{model_run}/ip"
files = [
  path + "/" + re.sub("(?<=\\=)(\\d+)$", "\\1/\\1.parquet", x)
  for x in os.listdir(path)
]

multi_results = pq.ParquetDataset(
  files,
  partitioning = partitioning,
  use_legacy_dataset = False
).read_pandas().to_pandas()

In [12]:
mr_counts = multi_results.groupby(["model_run"]).size()
mr_counts.head()

model_run
0    147707
1    152043
2    149472
3    152591
4    146150
dtype: int64

baseline counts vs. modelled counts

In [13]:
(len(ip.index), mr_counts.agg(["mean", "median", "min", "max"]).astype(int))

(128948,
 mean      148206
 median    147298
 min       144229
 max       155292
 dtype: int32)

In [14]:
mr_speldur = multi_results.groupby(["model_run"])["speldur"].agg("mean")
mr_speldur.head()

model_run
0    1.733371
1    1.737397
2    1.721379
3    1.707840
4    1.802908
Name: speldur, dtype: float64

baseline length of stay vs. modelled length of stay

In [15]:
(np.mean(ip.speldur), mr_speldur.agg(["mean", "median", "min", "max"]))

(2.1196528833328165,
 mean      1.722361
 median    1.721826
 min       1.581098
 max       1.828837
 Name: speldur, dtype: float64)